In [ ]:
%pip install yfinance
%pip install tqdm



In [3]:
import bs4 as bs
import requests
import yfinance as yf
import datetime
import numpy as np
from tqdm.notebook import tqdm, trange
import pandas as pd
import math
from dateutil.relativedelta import relativedelta




In [4]:
resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
soup = bs.BeautifulSoup(resp.text, 'lxml')
table = soup.find('table', {'class': 'wikitable sortable'})

def SP500_scraper(table):
  industries = {}
  tickers = []
  company_names = []
  company_industries = []
  #this iteration obtains all the tickers, company_names and industries which have tickers tagged to them
  for row in table.findAll('tr')[1:]:
      ticker = row.findAll('td')[0].text
      company_name = row.findAll('td')[1].text.strip()
      industry = row.findAll('td')[2].text
      tickers.append(ticker)
      company_names.append(company_name)
      company_industries.append(industry)
      
      #these 3 lines of code update the industries dictionary 
      #if industries.get(industry) == None:
        #industries[industry] = []
      #industries.get(industry).append(ticker)

  tickers = [ticker.replace('\n', '') for ticker in tickers]

  #these 2 lines of code update the industries dictionary with the tickers of the relevant companies of those industries
  #for industry in industries.keys():
    #industries[industry] = [ticker.replace('\n', '') for ticker in industries[industry]]

  #return Wikipedia Data pertaining to S&P_500_Companies
  holdings_df = pd.DataFrame({'Symbol': tickers, 'Company Name': company_names, 'Industry': company_industries})

  return holdings_df
  #print(holdings_df.head())
SP500_scraper(table)


,Symbol,Company Name,Industry
0,MMM,3M,Industrials
1,AOS,A. O. Smith,Industrials
2,ABT,Abbott,Health Care
3,ABBV,AbbVie,Health Care
4,ACN,Accenture,Information Technology
...,...,...,...
498,YUM,Yum! Brands,Consumer Discretionary
499,ZBRA,Zebra Technologies,Information Technology
500,ZBH,Zimmer Biomet,Health Care
501,ZION,Zions Bancorporation,Financials


In [6]:
def data_obtainer(tickers, month_delta, len_moving_window, data_start, data_end):
    data_end += relativedelta(months=-month_delta)
    data_start = data_end + relativedelta(months=-len_moving_window)

    df = yf.download(tickers, start=data_start, end=data_end)
    df = df[['Close']]
    df = df.apply(np.log).diff()

    return df


tickers = ['AAPL', 'MSFT', 'GOOGL']  # Example list of ticker symbols
month_delta = 3  # Example number of months to subtract from data_end
len_moving_window = 6  # Example length of the moving window
data_start = pd.to_datetime('2022-01-01')  # Example start date
data_end = pd.to_datetime('2022-12-31')  # Example end date

# Call the data_obtainer function with your parameters
result_df = data_obtainer(tickers, month_delta, len_moving_window, data_start, data_end)
print(result_df)


[*********************100%%**********************]  3 of 3 completed
               Close                    
                AAPL     GOOGL      MSFT
Date                                    
2022-03-30       NaN       NaN       NaN
2022-03-31 -0.017936 -0.020434 -0.017841
2022-04-01 -0.001720  0.007757  0.003594
2022-04-04  0.023417  0.019928  0.017778
2022-04-05 -0.019124 -0.016790 -0.013070
...              ...       ...       ...
2022-09-23 -0.015239 -0.014079 -0.012779
2022-09-26  0.002258 -0.005789 -0.001977
2022-09-27  0.006545 -0.006848 -0.004389
2022-09-28 -0.012732  0.025818  0.019520
2022-09-29 -0.050366 -0.026639 -0.014920

[127 rows x 3 columns]
